# Installs

In [ ]:
# Here we insatll the neccessary packahe versions

In [ ]:
!pip install -U "tensorflow-text==2.17.*"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 54.1 MB/s eta 0:00:00


In [ ]:
!pip install "tf-models-official==2.17.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 11.2 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=d97eec0a3881e1903758f5094cdc613303b9ba6d69d4ca216c7f97257af82706
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


# Imports

In [ ]:
# imports
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report
from matplotlib import pyplot as plt
import seaborn as sn


tf.get_logger().setLevel('ERROR')

# Version and gpu check

In [ ]:
# Ensure gpu is available for processing and that TensorFlow is of the right version

In [ ]:
tf.__version__
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# Function definitions

In [ ]:
# Read .txt files and split into paragraphs.
def get_split_text(input_file):
  f = open(input_file, "r")
  split = f.read().split('\n')

  return split

In [ ]:
# Encode paragraphs with applicable encoder model
def encoding(text, pre_proc, enc):

  text_preprocessed = pre_proc(text)
  text_encoded = enc(text_preprocessed)

  return text_encoded

# Data pre-processing

In [ ]:
# Check that input paragraphs correspond with labels provided

## Train

In [ ]:
path = "/content/drive/MyDrive/pan23-multi-author-analysis-dataset3-train"
dir_list = os.listdir(path)
num_all_train_files = int(len(dir_list)/2)
num_all_train_files

In [ ]:
# check for errors X
l_x = []
for F in tqdm(range(1,num_all_train_files+1)):
#for F in tqdm(range(1,1000+1)):
  url = '/content/drive/MyDrive/pan23-multi-author-analysis-dataset3-train/problem-'+str(F)+'.txt'
  split = get_split_text(url)

  l_x.append(len(split))

In [ ]:
l_x_rectified = [i-1 for i in l_x]

In [ ]:
# check for errors y
#for F in tqdm(range(1,1000+1)):
l_y = []

for F in tqdm(range(1,num_all_train_files+1)):
  url = '/content/drive/MyDrive/pan23-multi-author-analysis-dataset3-train/truth-problem-'+str(F)+'.json'
  with open(url, 'r') as file:
    data = json.load(file)
    l_y.append(len(data['changes']))

In [ ]:
# get erroneous indices
bad_index = [i for i in range(len(l_x)) if l_x_rectified[i] != l_y[i]]
bad_index

In [ ]:
#bad_index = [703, 3385]

In [ ]:
# remove erroneous indices
total_range = list(range(1,num_all_train_files+1))
print(len(total_range))
for i in bad_index:
  total_range.remove(i+1)
print(len(total_range))

## Test

In [ ]:
path = "/content/drive/MyDrive/pan23-multi-author-analysis-dataset3-validation"
dir_list = os.listdir(path)
num_all_test_files = int(len(dir_list)/2)
num_all_test_files

In [ ]:
# check for errors X
l_x_test = []
for F in tqdm(range(1,num_all_test_files+1)):
  url = '/content/drive/MyDrive/pan23-multi-author-analysis-dataset3-validation/problem-'+str(F)+'.txt'
  split = get_split_text(url)

  l_x_test.append(len(split))

In [ ]:
l_x_test_rectified = [i-1 for i in l_x_test]

In [ ]:
# check for errors y
l_y_test = []

for F in tqdm(range(1,num_all_test_files+1)):
  url = '/content/drive/MyDrive/pan23-multi-author-analysis-dataset3-validation/truth-problem-'+str(F)+'.json'
  with open(url, 'r') as file:
    data = json.load(file)
    l_y_test.append(len(data['changes']))

In [ ]:
# get erroneous indices
bad_index = [i for i in range(len(l_x_test)) if l_x_test_rectified[i] != l_y_test[i]]
bad_index

In [ ]:
#bad_index = []

In [ ]:
# remove erroneous indices
total_range_test = list(range(1,num_all_test_files+1))
print(len(total_range_test))
for i in bad_index:
  total_range_test.remove(i+1)
print(len(total_range_test))

# Create X_train and y_train

In [ ]:
# Set pre-processor and encoder model
# Encode every paragraph in input file and create difference vector and store in a usable format for TensorFlow
# Format labels for use with TensorFlow

##Small BERT - small_bert/bert_en_uncased_L-4_H-512_A-8

In [ ]:
small_bert_pre_process = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [ ]:
small_bert_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'

In [ ]:
small_bert_preprocess_model = hub.KerasLayer(small_bert_pre_process)
small_bert_model = hub.KerasLayer(small_bert_encoder)

In [ ]:
X_train_small_bert = []
for F in tqdm(total_range):
  url = '/content/drive/MyDrive/pan23-multi-author-analysis-dataset3-train/problem-'+str(F)+'.txt'
  split = get_split_text(url)
  text_encoded = encoding(split,small_bert_preprocess_model,small_bert_model)
  pooled = text_encoded['pooled_output']
  for i in range(1,len(pooled)):
    X_train_small_bert.append(pooled[i]-pooled[i-1])

In [ ]:
npX_train_small_bert = np.array(X_train_small_bert)

In [ ]:
len(npX_train_small_bert)

In [ ]:
y_train = []
for F in tqdm(total_range):
  url = '/content/drive/MyDrive/pan23-multi-author-analysis-dataset3-train/truth-problem-'+str(F)+'.json'
  with open(url, 'r') as file:
    data = json.load(file)
    y_train.extend(data['changes'])

In [ ]:
npy_train = np.array(y_train)

In [ ]:
len(npy_train)

## ALBERT

In [ ]:
albert_pre_process = 'https://tfhub.dev/tensorflow/albert_en_preprocess/3'

In [ ]:
albert_encoder = 'https://tfhub.dev/tensorflow/albert_en_base/2'

In [ ]:
albert_preprocess_model = hub.KerasLayer(albert_pre_process)
albert_model = hub.KerasLayer(albert_encoder)

In [ ]:
X_train_albert = []
for F in tqdm(total_range):
  url = '/content/drive/MyDrive/pan23-multi-author-analysis-dataset3-train/problem-'+str(F)+'.txt'
  split = get_split_text(url)
  text_encoded = encoding(split,albert_preprocess_model,albert_model)
  pooled = text_encoded['pooled_output']
  for i in range(1,len(pooled)):
    X_train_albert.append(pooled[i]-pooled[i-1])

In [ ]:
npX_train_albert = np.array(X_train_albert)

In [ ]:
len(npX_train_albert)

In [ ]:
len(npy_train)

## Small Electra

In [ ]:
small_electra_pre_process = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [ ]:
small_electra_encoder = 'https://tfhub.dev/google/electra_small/2'

In [ ]:
small_electra_preprocess_model = hub.KerasLayer(small_electra_pre_process)
small_electra_model = hub.KerasLayer(small_electra_encoder)

In [ ]:
X_train_electra = []
for F in tqdm(total_range):
  url = '/content/drive/MyDrive/pan23-multi-author-analysis-dataset3-train/problem-'+str(F)+'.txt'
  split = get_split_text(url)
  text_encoded = encoding(split,small_electra_preprocess_model,small_electra_model)
  pooled = text_encoded['pooled_output']
  for i in range(1,len(pooled)):
    X_train_electra.append(pooled[i]-pooled[i-1])

In [ ]:
npX_train_electra = np.array(X_train_electra)

In [ ]:
len(npX_train_electra)

In [ ]:
len(npy_train)

# Create X_test and y_test

In [ ]:
# Set pre-processor and encoder model
# Encode every paragraph in input file and create difference vector and store in a usable format for TensorFlow
# Format labels for use with TensorFlow

## Small BERT - small_bert/bert_en_uncased_L-4_H-512_A-8

In [ ]:
X_test_small_bert = []
for F in tqdm(total_range_test):
  url = '/content/drive/MyDrive/pan23-multi-author-analysis-dataset3-validation/problem-'+str(F)+'.txt'
  split = get_split_text(url)
  text_encoded = encoding(split,small_bert_preprocess_model,small_bert_model)
  pooled = text_encoded['pooled_output']
  for i in range(1,len(pooled)):
    X_test_small_bert.append(pooled[i]-pooled[i-1])

In [ ]:
npX_test_small_bert = np.array(X_test_small_bert)

In [ ]:
len(npX_test_small_bert)

In [ ]:
y_test = []
for F in tqdm(total_range_test):
  url = '/content/drive/MyDrive/pan23-multi-author-analysis-dataset3-validation/truth-problem-'+str(F)+'.json'
  with open(url, 'r') as file:
    data = json.load(file)
    y_test.extend(data['changes'])

In [ ]:
npy_test = np.array(y_test)

In [ ]:
len(npy_test)

## ALBERT

In [ ]:
X_test_albert = []
for F in tqdm(total_range_test):
  url = '/content/drive/MyDrive/pan23-multi-author-analysis-dataset3-validation/problem-'+str(F)+'.txt'
  split = get_split_text(url)
  text_encoded = encoding(split,albert_preprocess_model,albert_model)
  pooled = text_encoded['pooled_output']
  for i in range(1,len(pooled)):
    X_test_albert.append(pooled[i]-pooled[i-1])

In [ ]:
npX_test_albert = np.array(X_test_albert)

In [ ]:
len(npX_test_albert)

In [ ]:
len(npy_test)

## Small Electra

In [ ]:
X_test_electra = []
for F in tqdm(total_range_test):
  url = '/content/drive/MyDrive/pan23-multi-author-analysis-dataset3-validation/problem-'+str(F)+'.txt'
  split = get_split_text(url)
  text_encoded = encoding(split,small_electra_preprocess_model,small_electra_model)
  pooled = text_encoded['pooled_output']
  for i in range(1,len(pooled)):
    X_test_electra.append(pooled[i]-pooled[i-1])

In [ ]:
npX_test_electra = np.array(X_test_electra)

In [ ]:
len(npX_test_electra)

In [ ]:
len(npy_test)

# Classifier

In [ ]:
# Set metrics to show while training
# Specify neural network classifier infrastructure
# Train network
# Perform prediction and provide summary of results

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

## Small BERT

In [ ]:
model_small_bert = tf.keras.Sequential([
    tf.keras.layers.Dropout(0.1, name="dropout"),
    tf.keras.layers.Dense(1, activation='sigmoid', name="output")
])

In [ ]:
model_small_bert.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
history_small_bert = model_small_bert.fit(npX_train_small_bert, npy_train, epochs=15)

In [ ]:
y_pred_bert = model_small_bert.predict(npX_test_small_bert)
y_pred_bert = y_pred_bert.flatten()
y_pred_bert = np.where(y_pred_bert > 0.5, 1, 0)

In [ ]:
cm_small_bert = confusion_matrix(y_test, y_pred_bert)

In [ ]:
sn.heatmap(cm_small_bert, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_pred_bert))

              precision    recall  f1-score   support

           0       0.54      0.68      0.60      2159
           1       0.50      0.35      0.41      1953

    accuracy                           0.52      4112
   macro avg       0.52      0.51      0.50      4112
weighted avg       0.52      0.52      0.51      4112



## Albert

In [ ]:
model_albert = tf.keras.Sequential([
    tf.keras.layers.Dropout(0.1, name="dropout"),
    tf.keras.layers.Dense(1, activation='sigmoid', name="output")
])

In [ ]:
model_albert.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
history_albert = model_albert.fit(npX_train_albert, npy_train, epochs=15)

In [ ]:
y_pred_albert = model_albert.predict(npX_test_albert)
y_pred_albert = y_pred_albert.flatten()
y_pred_albert = np.where(y_pred_albert > 0.5, 1, 0)

In [ ]:
cm_albert = confusion_matrix(y_test, y_pred_albert)

In [ ]:
sn.heatmap(cm_albert, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_pred_albert))

              precision    recall  f1-score   support

           0       0.55      0.70      0.62      2159
           1       0.53      0.38      0.44      1953

    accuracy                           0.55      4112
   macro avg       0.54      0.54      0.53      4112
weighted avg       0.54      0.55      0.53      4112



## Electra

In [ ]:
model_electra = tf.keras.Sequential([
    tf.keras.layers.Dropout(0.1, name="dropout"),
    tf.keras.layers.Dense(1, activation='sigmoid', name="output")
])

In [ ]:
model_electra.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
history_electra = model_electra.fit(npX_train_electra, npy_train, epochs=15)

In [ ]:
y_pred_electra = model_electra.predict(npX_test_electra)
y_pred_electra = y_pred_electra.flatten()
y_pred_electra = np.where(y_pred_electra > 0.5, 1, 0)

In [ ]:
cm_electra = confusion_matrix(y_test, y_pred_electra)

In [ ]:
sn.heatmap(cm_electra, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_pred_electra))

              precision    recall  f1-score   support

           0       0.54      0.75      0.63      2159
           1       0.52      0.30      0.38      1953

    accuracy                           0.53      4112
   macro avg       0.53      0.52      0.50      4112
weighted avg       0.53      0.53      0.51      4112

